In [1]:
# In this example, you'll be creating a Python script to visualize the weather of 500+ cities across the world, 
# at varying distances from the equator. To accomplish this, you'll be utilizing a simple Python library, 
# the OpenWeatherMap API, and a little common sense to create a representative model of weather across world cities.
# The first requirement is to create a series of scatter plots to showcase the following relationships:

# Temperature (F) vs. Latitude
# Humidity (%) vs. Latitude
# Cloudiness (%) vs. Latitude
# Wind Speed (mph) vs. Latitude

# After each plot, add a sentence or two explaining what the code is analyzing.
# The second requirement is to run linear regression on each relationship. This time, separate the plots 
# into Northern Hemisphere (greater than or equal to 0 degrees latitude) and Southern Hemisphere (less than 0 
# degrees latitude):

# Northern Hemisphere - Temperature (F) vs. Latitude
# Southern Hemisphere - Temperature (F) vs. Latitude
# Northern Hemisphere - Humidity (%) vs. Latitude
# Southern Hemisphere - Humidity (%) vs. Latitude
# Northern Hemisphere - Cloudiness (%) vs. Latitude
# Southern Hemisphere - Cloudiness (%) vs. Latitude
# Northern Hemisphere - Wind Speed (mph) vs. Latitude
# Southern Hemisphere - Wind Speed (mph) vs. Latitude

# After each pair of plots, take the time to explain what the linear regression is modeling. 
# For example, describe any relationships you notice and any other analysis you may have.
# Your final notebook must:

# Randomly select at least 500 unique (non-repeat) cities based on latitude and longitude.
# Perform a weather check on each of the cities using a series of successive API calls.
# Include a print log of each city as it's being processed with the city number and city name.
# Save a CSV of all retrieved data and a PNG image for each scatter plot.

In [2]:
# Import dependencies:

import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
from scipy.stats import linregress
import numpy as np
from config import api_key
import time   #used for time.sleep

In [3]:
# Printed api key to ensure that it imported correctly
# print(api_key)

In [4]:
# Generate a random list of 500 unique cities with a reasonably even distribution across the globe

# Set a range for lat and lng that includes the whole Earth
# latitude range is -90 to 90, longitude range is -180 to 180
lats = []
longs = []

lats = np.random.uniform(-90, 90, 1300)
longs = np.random.uniform(-180, 180, 1300)

#print(lats)
#print(longs)

In [5]:
# zip lists together to form coordinates
lat_long = zip(lats, longs)
lat_long

In [6]:
#lat_long = pd.DataFrame(lat_long)
#lat_long

In [7]:
# Use citipy to find cities near coordinates

#Import citipy
from citipy import citipy

cities = []
countries = []

for lat_long in lat_long:
    city = citipy.nearest_city(lat_long[0], lat_long[1]).city_name
    country = citipy.nearest_city(lat_long[0], lat_long[1]).country_code
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        countries.append(country)

# Print the city count to confirm sufficient count
print(len(cities))
print(len(countries))

541
541


In [17]:
cities = [city.title() for city in cities]
countries = [country.upper() for country in countries]

In [18]:
# Build dataframe to hold data
weather_df = pd.DataFrame({"Cities" : cities, "Countries" : countries})
weather_df.head()

,Cities,Countries
0,Mataura,PF
1,Lorengau,PG
2,Rikitea,PF
3,Umm Kaddadah,SD
4,Olinda,BR


In [22]:
# Check for NaNs
#weather_df.isnull().sum()

In [24]:
# Add places to hold data for each city

weather_df["Latitude"] = ""
weather_df["Longitude"] = ""
weather_df["Temperature (F)"] = ""
weather_df["Humidity"] = ""
weather_df["Cloudiness"] = ""
weather_df["Wind Speed (mph)"] = ""
weather_df.head()

,Cities,Countries,Latitude,Longitude,Temperature (F),Humidity,Cloudiness,Wind Speed (mph)
0,Mataura,PF,,,,,,
1,Lorengau,PG,,,,,,
2,Rikitea,PF,,,,,,
3,Umm Kaddadah,SD,,,,,,
4,Olinda,BR,,,,,,


In [33]:
# Use OpenWeather to get weather data for generated list of cities, test on one city

base_url = "http://api.openweathermap.org/data/2.5/weather?"
city_name = "Minneapolis"
country_code = "US"
query_url = f"{base_url}q={city_name},{country_code}&appid={api_key}&units=imperial"
#print(query_url) - test url worked
minneapolis_weather = requests.get(query_url).json()
minneapolis_weather

{'coord': {'lon': -93.2638, 'lat': 44.98},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01n'}],
 'base': 'stations',
 'main': {'temp': -10.77,
  'feels_like': -21.51,
  'temp_min': -11.2,
  'temp_max': -9.4,
  'pressure': 1022,
  'humidity': 64},
 'visibility': 10000,
 'wind': {'speed': 6.91, 'deg': 300},
 'clouds': {'all': 1},
 'dt': 1612678290,
 'sys': {'type': 1,
  'id': 4900,
  'country': 'US',
  'sunrise': 1612704302,
  'sunset': 1612740567},
 'timezone': -21600,
 'id': 5037649,
 'name': 'Minneapolis',
 'cod': 200}

In [34]:
#test get lat
minneapolis_weather["coord"]["lat"]

44.98

In [35]:
#test get long
minneapolis_weather["coord"]["lon"]

-93.2638

In [37]:
#test get temp
minneapolis_weather["main"]["temp"]

-10.77

In [40]:
#test get humidity
minneapolis_weather["main"]["humidity"]

64

In [46]:
#test get cloudiness
minneapolis_weather["clouds"]["all"]
#cloudiness exists on a scale documented at https://openweathermap.org/weather-conditions

1

In [49]:
#test get windspeed
minneapolis_weather["wind"]["speed"]

6.91

In [27]:
my_params = {"key" : api_key, "units" : imperial}

for index, row in weather_df.iterrows():
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    city = row["Cities"]
    country = row["Countries"]
    
    my_params[city_name] = city
    my_params[country_code] = country
    
    #make request
    cities_weather = requests.get(base_url, params=my_params)
    time.sleep(1)
    